In [1]:
using Pkg;

In [2]:
FLAG = true;
if ! @isdefined(PDDL) && FLAG
    Pkg.add("PDDL")
    Pkg.add("PDDLViz")
    Pkg.add("GLMakie")
    Pkg.add("PlanningDomains")
    Pkg.add(url="https://github.com/JuliaPlanners/SymbolicPlanners.jl")
else
    println("Skipped reinstallation")
end;

    Updating registry at `/opt/julia/registries/General.toml`
   Resolving package versions...
   Installed ValSplit ──────────── v0.1.1
   Installed ExprTools ─────────── v0.1.10
   Installed AutoHashEquals ────── v2.2.0
   Installed Julog ─────────────── v0.1.15
   Installed DocStringExtensions ─ v0.9.3
   Installed PDDL ──────────────── v0.2.18
   Installed ParserCombinator ──── v2.2.1
    Updating `/opt/julia/environments/v1.9/Project.toml`
  [2c8894f9] + PDDL v0.2.18
    Updating `/opt/julia/environments/v1.9/Manifest.toml`
  [15f4f7f2] + AutoHashEquals v2.2.0
  [ffbed154] + DocStringExtensions v0.9.3
  [e2ba6199] + ExprTools v0.1.10
  [5d8bcb5e] + Julog v0.1.15
  [2c8894f9] + PDDL v0.2.18
  [fae87a5f] + ParserCombinator v2.2.1
  [0625e100] + ValSplit v0.1.1
Precompiling project...
  ✓ AutoHashEquals
  ✓ ExprTools
  ✓ DocStringExtensions
  ✓ ValSplit
  ✓ Julog
  ✓ ParserCombinator
  ✓ PDDL
  7 dependencies successfully precompiled in 23 seconds. 61 already precompiled.
   Resolvin

In [40]:
using PDDL, PlanningDomains, SymbolicPlanners

In [185]:
compounds = initial.types
items = ["ingredient", "tool", "dish"]
items_instances = []
locations_instances = []
status_instances = []

# Iterate over the Set and extract the type and value
for compound in compounds
    # Extract type and value by splitting at '(' and ')'
    TYPE = compound.name
    INSTANCE = compound.args[1].name # Julia arrays are 1-indexed, so args[2] refers to the second element
    println(INSTANCE)

    if TYPE in items
        push!(items_instances, INSTANCE)
    elseif TYPE == "status"
        push!(status_instances, INSTANCE)
    elseif TYPE == "location"
        push!(locations_instances, INSTANCE)
    end
end

all_combinations = [] # All combinations of the three extracted lists adding ":" at the beginning of the string
println(items_instances)
# Generate all combinations
for comb in Iterators.product(items_instances, status_instances, locations_instances)
    println(comb)
    comb_str = join(comb, ":")
    push!(all_combinations, comb_str)

    # Check and update the initial values dictionary
    if !haskey(initial.values["item-count"], comb_str)
        initial.values["item-count"][comb_str] = 0
    end
end


dirty
sushi
fish
sushitool
sa
raw
pa
unprepared
prepared
rice
clean
seaweed
prepare
robot1
cooked
Any[]


In [178]:
compounds = initial.types
items = ["ingredient", "tool", "dish"]
items_instances = []
locations_instances = []
status_instances = []
# Iterate over the Set and extract the type and value
for compound in compounds
    # Extract type and value by splitting at '(' and ')'
    TYPE = compound.name
    INSTANCE = compound.args[1].symbol
    println(INSTANCE)
    if TYPE in items
        items_instances.append(value)
    if TYPE=="status"
        status_instances.append(value)
    if TYPE=="location"
        locations_instances.append(value)
end
all_combination = [] #All combinatiuons of the three stracted lists adding ":" at the beggining of the string
for comb in all_combinations
    if not comb in initial.values["item-count"]:
        initial.values["item-count"][comb] = 0
        
end

LoadError: syntax: line break in ":" expression

In [183]:
initial = initstate(a_domain, a_problem)
data = initial.values
for (key, inner_dict) in data
    println("Key: ", key)
    println("Items:")
    for (inner_key_tuple, value) in inner_dict
        println("  - Items: ", inner_key_tuple, typeof(inner_key_tuple), " => Count: ", value)
    end
    println()  # Adding a newline for better readability
end

Key: item-count
Items:
  - Items: (:sushitool, :clean, :sa)Tuple{Symbol, Symbol, Symbol} => Count: 1
  - Items: (:sushi, :unprepared, :pa)Tuple{Symbol, Symbol, Symbol} => Count: 1
  - Items: (:rice, :raw, :sa)Tuple{Symbol, Symbol, Symbol} => Count: 1
  - Items: (:fish, :raw, :sa)Tuple{Symbol, Symbol, Symbol} => Count: 1
  - Items: (:seaweed, :clean, :sa)Tuple{Symbol, Symbol, Symbol} => Count: 0
  - Items: (:seaweed, :raw, :sa)Tuple{Symbol, Symbol, Symbol} => Count: 1



In [199]:
a_domain = load_domain( "domain.pddl" )
a_problem = load_problem( "problem.pddl" )
initial = initstate(a_domain, a_problem)
the_goals = MinStepsGoal( a_problem )
a_planner = AStarPlanner(HAdd())
solved = a_planner( a_domain, initial, the_goals)

PathSearchSolution{...}
  status: :success
  plan: 17-element Vector{Term}
    (move robot1 pa sa)
    (pick-up-item robot1 sushitool clean sa)
    (move robot1 sa pa)
    (drop-item robot1 sushitool clean pa)
    (move robot1 pa sa)
    (pick-up-item robot1 fish raw sa)
    (move robot1 sa pa)
    (drop-item robot1 fish raw pa)
    (move robot1 pa sa)
    (pick-up-item robot1 seaweed raw sa)
    (move robot1 sa pa)
    (drop-item robot1 seaweed raw pa)
    (move robot1 pa sa)
    (pick-up-item robot1 rice raw sa)
    (move robot1 sa pa)
    (drop-item robot1 rice raw pa)
    (to-process robot1 prepare sushi pa)
  trajectory: 18-element Vector{GenericState}
  expanded: 86
  search_tree: Dict{UInt64, SymbolicPlanners.PathNode{GenericState}} with 119 entries
  search_frontier: DataStructures.PriorityQueue{UInt64, Tuple{Float32, Float32, Int64}, Base.Order.ForwardOrdering} with 33 entries
  search_order: 86-element Vector{UInt64}

In [200]:
# Open a file for writing
open("tree.txt", "w") do file
    for node in solved.search_order
        # Write each line to the file
        println(file, " ", solved.search_tree[node])
        println(file, "\n\n")
    end
end

In [194]:
solved.search_tree

Dict{UInt64, SymbolicPlanners.PathNode{GenericState}} with 56 entries:
  0x501a66e116051511 => PathNode{GenericState}(0x501a66e116051511, GenericState…
  0xa25cbbb159cd2467 => PathNode{GenericState}(0xa25cbbb159cd2467, GenericState…
  0x7457f4eeacaf7118 => PathNode{GenericState}(0x7457f4eeacaf7118, GenericState…
  0x1ba3443d742cb0c5 => PathNode{GenericState}(0x1ba3443d742cb0c5, GenericState…
  0x28366157455dd594 => PathNode{GenericState}(0x28366157455dd594, GenericState…
  0xd0141b3a76a4dfea => PathNode{GenericState}(0xd0141b3a76a4dfea, GenericState…
  0xd14c0412a5e9e9ff => PathNode{GenericState}(0xd14c0412a5e9e9ff, GenericState…
  0x910d7cac9b32cc84 => PathNode{GenericState}(0x910d7cac9b32cc84, GenericState…
  0xfd3b72c5a344e2fd => PathNode{GenericState}(0xfd3b72c5a344e2fd, GenericState…
  0x758d42a875e5beeb => PathNode{GenericState}(0x758d42a875e5beeb, GenericState…
  0x77c13058143255c6 => PathNode{GenericState}(0x77c13058143255c6, GenericState…
  0x7acb66c3c8b38b66 => PathNode{Gener

In [193]:
for node in solved.search_order
    println(" ", solved.search_tree[node])
    println("\n\n")
end;

 SymbolicPlanners.PathNode{GenericState}(0xd23eb6a2a0d4b8b7, GenericState(Set(Compound[status(dirty), dish(sushi), ingredient(fish), tool(sushitool), location(sa), status(raw), location(pa), status(unprepared), status(prepared), ingredient(rice), status(clean), ingredient(seaweed), process(prepare), robot(robot1), status(cooked)]), Set(Term[connected(sa, pa), connected(pa, sa), applicable(dirty, sushitool), applicable(raw, rice), item-at(rice, raw, sa), applicable(unprepared, sushi), applicable(clean, sushitool), item-at(seaweed, raw, sa), item-at(fish, raw, sa), applicable(cooked, fish), item-at(sushi, unprepared, pa), robot-free(robot1), applicable(raw, fish), applicable(cooked, rice), applicable(raw, seaweed), applicable(cooked, seaweed), requires-item(prepare, sushi, rice, raw), requires-item(prepare, sushi, sushitool, clean), applicable(prepared, sushi), robot-at(robot1, pa), requires-item(prepare, sushi, fish, raw), requires-item(prepare, sushi, seaweed, raw), done-at(prepare, su

In [195]:
for (index, node) in enumerate(solved.search_order)
    current_node = solved.search_tree[node]
    previous_node_ref = current_node.previous_ref
    action_taken = previous_node_ref ? previous_node_ref.action : "Root Node (No Action)"
    
    println("Node $(index):")
    println("    ID: ", node)
    println("    Previous Node: ", previous_node_ref ? previous_node_ref.id : "None")
    println("    Action Taken: ", action_taken)
    println("    State: ", current_node.state)
    println("\n")
end


LoadError: type PathNode has no field previous_ref

In [ ]:
solved.search_tree[0x6e2e8f67bb9bd060]

In [125]:
println("Whole plan: (", length(solved), ") steps")
for s in collect(solved)
    println(" ", s)
end;

Whole plan: (12) steps
 move(robot1, pa, sa)
 pick-up(robot1, seaweed, sa)
 move(robot1, sa, pa)
 use-ingredient(robot1, seaweed, pa)
 move(robot1, pa, sa)
 pick-up(robot1, rice, sa)
 move(robot1, sa, pa)
 use-ingredient(robot1, rice, pa)
 move(robot1, pa, sa)
 pick-up(robot1, fish, sa)
 move(robot1, sa, pa)
 use-ingredient(robot1, fish, pa)
